In [3]:
import sys
import os
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt
import arviz as az
import joblib
from sklearn.preprocessing import StandardScaler

# Add the baseball_simulator directory to path
sys.path.append('./baseball_simulator')

# Import configuration and data processing modules
import config
from data_fetcher import fetch_statcast_data, fetch_defensive_stats, fetch_park_factors
from data_processor import (
    process_statcast_data,
    create_helper_columns,
    calculate_batter_daily_totals,
    calculate_pitcher_daily_totals,
    calculate_ballasted_batter_stats,
    calculate_ballasted_pitcher_stats,
    join_together_final_df
)

2025-06-09 13:09:18,726 - WARNING - configdefaults - g++ not available, if using conda: `conda install gxx`


2025-06-09 13:09:18,726 - WARNING - configdefaults - g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
2025-06-09 13:09:33,377 - INFO - preview - arviz_base not installed
2025-06-09 13:09:33,387 - INFO - preview - arviz_stats not installed
2025-06-09 13:09:33,393 - INFO - preview - arviz_plots not installed


In [5]:
def prepare_training_data():
    """
    Prepare training data using the exact same pipeline as the simulator.
    Only uses 2023-2024 data for model fitting.
    """
    print("Starting data preparation...")
    
    # Fetch raw data (2021-2024 for ballasting, but will filter to 2023-2024 for training)
    print("Fetching statcast data...")
    statcast_df = fetch_statcast_data()
    
    print("Fetching defensive stats...")
    defensive_df = fetch_defensive_stats(2021, 2024)  # Fetching defensive stats for 2021-2024
    
    print("Fetching park factors...")
    park_factors_df = fetch_park_factors()
    
    # Process statcast data
    print("Processing statcast data...")
    processed_df = process_statcast_data(statcast_df)
    
    # Create helper columns
    print("Creating helper columns...")
    processed_df = create_helper_columns(processed_df)
    
    # Calculate daily totals
    print("Calculating batter daily totals...")
    batter_daily_df = calculate_batter_daily_totals(processed_df)
    
    print("Calculating pitcher daily totals...")
    pitcher_daily_df = calculate_pitcher_daily_totals(processed_df)
    
    # Calculate ballasted stats
    print("Calculating ballasted batter stats...")
    ballasted_batter_df = calculate_ballasted_batter_stats(batter_daily_df)
    
    print("Calculating ballasted pitcher stats...")
    ballasted_pitcher_df = calculate_ballasted_pitcher_stats(pitcher_daily_df)
    
    # Join everything together
    print("Joining final dataset...")
    final_df = join_together_final_df(
        processed_df, 
        ballasted_batter_df, 
        ballasted_pitcher_df, 
        defensive_df, 
        park_factors_df
    )
    
    # Filter to training requirements
    print("Applying training filters...")
    
    # Filter to innings 1-3 only
    final_df = final_df[final_df['inning'] <= 3].copy()
    
    # Filter to 2023-2024 for training (as specified by user)
    training_years = [2023, 2024]
    final_df = final_df[final_df['game_year'].isin(training_years)].copy()
    
    # Remove rows with missing values in predictor columns
    final_df = final_df.dropna(subset=config.PREDICTOR_COLS + ['outcome_cat']).copy()
    
    print(f"Final training dataset shape: {final_df.shape}")
    print(f"Training years: {sorted(final_df['game_year'].unique())}")
    print(f"Outcome distribution:\n{final_df['outcome_cat'].value_counts().sort_index()}")
    
    return final_df


final_df = prepare_training_data()
final_df.head()

Starting data preparation...
Fetching statcast data...
start_dt 2025-06-08
end_dt 2025-06-09
This is a large query, it may take a moment to complete


 50%|█████     | 1/2 [00:00<00:00,  1.92it/s]c:\Users\samba\OneDrive\Python_Projects\first_3_innings_model\.venv\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
100%|██████████| 2/2 [00:02<00:00,  1.04s/it]

Fetching defensive stats...



c:\Users\samba\OneDrive\Python_Projects\first_3_innings_model\.venv\Lib\site-packages\pybaseball\statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)


TypeError: fetch_defensive_stats() takes 1 positional argument but 2 were given

In [ ]:


def prepare_model_inputs(final_df):
    """
    Prepare model inputs ensuring consistency with simulator expectations.
    """
    print("Preparing model inputs...")
    
    # Extract features in the exact order expected by simulator
    X = final_df[config.PREDICTOR_COLS].to_numpy()
    y = final_df['outcome_cat'].to_numpy()
    
    # Separate continuous and categorical features
    n_continuous = len(config.CONTINUOUS_COLS)
    X_continuous = X[:, :n_continuous]
    X_categorical = X[:, n_continuous:]
    
    # Scale only continuous features
    print("Scaling continuous features...")
    scaler = StandardScaler()
    X_continuous_scaled = scaler.fit_transform(X_continuous)
    
    # Combine scaled continuous with unscaled categorical
    X_final = np.concatenate([X_continuous_scaled, X_categorical], axis=1)
    
    print(f"Feature matrix shape: {X_final.shape}")
    print(f"Continuous features: {X_continuous.shape[1]}")
    print(f"Categorical features: {X_categorical.shape[1]}")
    print(f"Target categories: {len(np.unique(y))}")
    
    return X_final, y, scaler

def build_and_train_model(X, y):
    """
    Build and train the PyMC multinomial logistic regression model.
    """
    print("Building PyMC model...")
    
    n_obs, n_predictors = X.shape
    n_categories = config.N_CATEGORIES
    
    print(f"Model dimensions:")
    print(f"  Observations: {n_obs}")
    print(f"  Predictors: {n_predictors}")
    print(f"  Categories: {n_categories}")
    
    with pm.Model() as multi_outcome_model:
        # Intercepts (n_categories - 1 free parameters, last category = 0 for identifiability)
        intercepts_offset = pm.Normal(
            "intercepts_offset", 
            mu=0, 
            sigma=1.5, 
            shape=(n_categories - 1,)
        )
        intercepts = pm.Deterministic(
            "intercepts", 
            pt.concatenate([intercepts_offset, pt.zeros(1)])
        )
        
        # Coefficients (n_predictors × (n_categories - 1) free parameters)
        betas_offset = pm.Normal(
            "betas_offset", 
            mu=0, 
            sigma=1.0, 
            shape=(n_predictors, n_categories - 1)
        )
        betas = pm.Deterministic(
            "betas", 
            pt.concatenate([betas_offset, pt.zeros((n_predictors, 1))], axis=1)
        )
        
        # Linear predictor
        mu = intercepts + X @ betas
        
        # Softmax to get probabilities
        p = pm.math.softmax(mu, axis=1)
        
        # Likelihood
        outcome = pm.Categorical("outcome", p=p, observed=y)
    
    # Sample from the model
    print("Starting MCMC sampling...")
    print("This may take several minutes...")
    
    with multi_outcome_model:
        # Use JAX backend for faster sampling if available
        try:
            idata = pm.sample(
                draws=1000,
                tune=1000,
                chains=2,
                target_accept=0.90,
                nuts_sampler="numpyro",
                random_seed=42
            )
            print("Used JAX/numpyro backend for sampling")
        except Exception as e:
            print(f"JAX backend failed ({e}), falling back to default backend")
            idata = pm.sample(
                draws=1000,
                tune=1000,
                chains=2,
                target_accept=0.90,
                random_seed=42
            )
    
    return multi_outcome_model, idata

def validate_model_convergence(idata):
    """
    Validate model convergence using ArviZ diagnostics.
    """
    print("Validating model convergence...")
    
    # Check R-hat values
    r_hat = az.rhat(idata)
    max_r_hat = float(r_hat.max().values)
    print(f"Maximum R-hat: {max_r_hat:.4f}")
    
    if max_r_hat > 1.1:
        print("WARNING: Some R-hat values > 1.1, model may not have converged")
    else:
        print("✓ All R-hat values < 1.1, good convergence")
    
    # Check effective sample size
    ess = az.ess(idata)
    min_ess = float(ess.min().values)
    print(f"Minimum effective sample size: {min_ess:.0f}")
    
    if min_ess < 400:
        print("WARNING: Some ESS values < 400, consider longer sampling")
    else:
        print("✓ All ESS values > 400, sufficient sampling")
    
    # Summary statistics
    print("\nModel summary:")
    print(az.summary(idata, round_to=4))
    
    return max_r_hat < 1.1 and min_ess >= 400

def save_model_artifacts(idata, scaler):
    """
    Save model and scaler with the exact filenames expected by ModelLoader.
    """
    print("Saving model artifacts...")
    
    # Save PyMC inference data
    model_filename = "multi_outcome_model.nc"
    idata.to_netcdf(model_filename)
    print(f"✓ Saved model to: {model_filename}")
    
    # Save scaler
    scaler_filename = "pa_outcome_scaler.joblib"
    joblib.dump(scaler, scaler_filename)
    print(f"✓ Saved scaler to: {scaler_filename}")
    
    return model_filename, scaler_filename

def test_model_loading():
    """
    Test that the saved model can be loaded with the existing ModelLoader.
    """
    print("Testing model loading...")
    
    try:
        from model_loader import ModelLoader
        
        # Test loading
        loader = ModelLoader(base_dir="./")
        model, scaler = loader.load_all()
        
        print("✓ Model and scaler loaded successfully with ModelLoader")
        
        # Test scaler dimensions
        dummy_continuous = np.random.randn(5, len(config.CONTINUOUS_COLS))
        scaled = scaler.transform(dummy_continuous)
        
        if scaled.shape == dummy_continuous.shape:
            print("✓ Scaler transforms data correctly")
        else:
            print("✗ Scaler dimension mismatch")
            
    except Exception as e:
        print(f"✗ Error testing model loading: {e}")

def main():
    """
    Main function to refit the PyMC model.
    """
    print("="*60)
    print("REFITTING PYMC MODEL FOR BASEBALL SIMULATOR")
    print("="*60)
    
    # Validate configuration
    print(f"Using predictor columns: {len(config.PREDICTOR_COLS)} features")
    print(f"Continuous features: {len(config.CONTINUOUS_COLS)}")
    print(f"Categorical features: {len(config.CATEGORICAL_COLS)}")
    print(f"Number of outcome categories: {config.N_CATEGORIES}")
    print(f"Training on years: 2023-2024")
    print()
    
    try:
        # Step 1: Prepare training data
        final_df = prepare_training_data()
        
        # Step 2: Prepare model inputs
        X, y, scaler = prepare_model_inputs(final_df)
        
        # Step 3: Build and train model
        model, idata = build_and_train_model(X, y)
        
        # Step 4: Validate convergence
        converged = validate_model_convergence(idata)
        
        # Step 5: Save artifacts
        model_file, scaler_file = save_model_artifacts(idata, scaler)
        
        # Step 6: Test loading
        test_model_loading()
        
        print()
        print("="*60)
        if converged:
            print("✓ MODEL REFITTING COMPLETED SUCCESSFULLY")
        else:
            print("⚠ MODEL REFITTING COMPLETED WITH CONVERGENCE WARNINGS")
        print("="*60)
        print(f"Model saved to: {model_file}")
        print(f"Scaler saved to: {scaler_file}")
        print()
        print("The model is now ready for use with the baseball simulator.")
        
    except Exception as e:
        print(f"✗ Error during model refitting: {e}")
        raise

if __name__ == "__main__":
    main()